<a href="https://colab.research.google.com/github/MaxVortman/IDEA-code-clones/blob/master/alex/code_clones_ast_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/My\ Drive/code-clones/

In [ ]:
DATA_DIR = "data"
XML = "duplicates_marked.xml"
PROJECT = "Jenkins" #just put another one

projPath = os.path.join(DATA_DIR, PROJECT)
xmlPath = os.path.join(projPath, XML)

duplicates = xmlextractor.parseXml(xmlPath, projPath)

In [ ]:
%cd /content

In [ ]:
import treelstm
import javaToPythonTreeNormalizerMiddleware

In [ ]:
!mv models/ treelstm/

In [ ]:
%cd treelstm/

In [ ]:
import xmlextractor
import os
from tqdm import tqdm
import pexpect
import sys
import time
import numpy as np
import json

#спасибо Максиму за офигенные лайфхаки с консолью, Ctrl C + Ctrl V
def expectCodeVectors(child, acc):
  index1 = child.expect(["Code vector:", "Modify the file: \"Input.java\" and press any key when ready, or \"q\" / \"quit\" / \"exit\" to exit"])
  if index1 == 0:
    index2 = child.expect(["Modify the file: \"Input.java\" and press any key when ready, or \"q\" / \"quit\" / \"exit\" to exit", "Original name:"])
    acc.append(child.before)
    if index2 == 0:
      return acc
    elif index2 == 1:
      return expectCodeVectors(child, acc)
  elif index1 == 1:
    return []

def list2vec(l):
  result_acc = []
  for code_vector in acc:
    result_code_vector = []
    for s in code_vector:
      result_code_vector.append(float(s))
    result_acc.append(result_code_vector)
  return result_acc

child = pexpect.spawn ('python3 treelstm.py --load models/java_for_lstm_prepared/defaultModel --export --pred', timeout=None)

code_vectors = []
child.expect("Modify the file: \"Input.java\" and press any key when ready, or \"q\" / \"quit\" / \"exit\" to exit")

for duplicate in tqdm(duplicates, ascii=True):
  duplicate_code_vectors = []
  for fragment in duplicate.fragments:
    with open("Input.java", 'w') as f:
      child = pexpect.spawn ('python3 javaToPythonTreeNormalizer.py --process models/java_for_lstm', timeout=None)
      f.write(fragment)
    child.sendline("r")
    acc = expectCodeVectors(child, [])
    if acc == []:
      continue
    acc = list2vec(acc)
    duplicate_code_vectors.append(np.mean(acc, axis=0).tolist())
  code_vectors.append({'exp': duplicate.exp, 'vectors': duplicate_code_vectors})

child.sendline("q")
child.expect(pexpect.EOF)
child.close()

In [ ]:
JSON_FILE = '/content/gdrive/My Drive/code-clones/alex/csv/ast_lstm_vectors.json'

with open(JSON_FILE, 'w') as outfile:
    json.dump(code_vectors, outfile)

In [ ]:
#OK